# Modules update
## Notebook to update current just-dna-seq modules

In [6]:
from pycomfort import files
from pathlib import Path
base = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
data = base / "data"
modules = data / "modules"
files.tprint(modules)







modules
	just_longevitymap
		longevitymap.sqlite


In [2]:
import sqlite3
import polars as pl

# Connect to the database
db_path = modules / "just_longevitymap" / "longevitymap.sqlite"
conn = sqlite3.connect(db_path)

# Get list of all tables in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(f"  - {table[0]}")



Tables in the database:
  - gene
  - population
  - variant
  - allele_weights
  - categories


In [3]:
# Get schema for each table
for table in tables:
    table_name = table[0]
    print(f"\nSchema for table '{table_name}':")
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    for col in columns:
        print(f"  {col[1]} ({col[2]})")




Schema for table 'gene':
  id (INTEGER)
  name (TEXT)
  symbol (TEXT)
  alias (TEXT)
  description (TEXT)
  omim (TEXT)
  ensembl (TEXT)
  uniprot (TEXT)
  unigene (TEXT)
  cytogenetic_location (TEXT)

Schema for table 'population':
  id (INTEGER)
  name (TEXT)

Schema for table 'variant':
  id (INTEGER)
  location (TEXT)
  study_design (TEXT)
  conclusions (TEXT)
  association (TEXT)
  gender (TEXT)
  quickref (TEXT)
  quickyear (INTEGER)
  quickpubmed (TEXT)
  identifier (TEXT)
  gene_id (INTEGER)
  population_id (INTEGER)

Schema for table 'allele_weights':
  id (INTEGER)
  allele (TEXT)
  state (TEXT)
  zygosity (TEXT)
  weight (REAL)
  rsid (TEXT)
  priority (TEXT)
  category_id (INTEGER)

Schema for table 'categories':
  id (INTEGER)
  name (TEXT)


In [4]:
# Get row counts for each table
for table in tables:
    table_name = table[0]
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    print(f"{table_name}: {count} rows")



gene: 886 rows
population: 82 rows
variant: 3476 rows
allele_weights: 1043 rows
categories: 12 rows


## Join with rsid

In [5]:
import polars as pl

# Get the Ensembl cache path and scan SNV parquets
ensembl_cache = download_ensembl_annotations()
snv_dir = ensembl_cache / "data" / "SNV"
if not snv_dir.exists():
    snv_dir = ensembl_cache / "SNV"

# Scan all parquet files in the directory as one lazy frame
df_ensembl = pl.scan_parquet(snv_dir)

ModuleNotFoundError: No module named 'just_dna_pipelines.annotation.ensembl_annotations'

In [ ]:
df_ensembl.head(5).collect()

chrom,start,end,id,ref,alt,qual,filter,cosmic_101,clinvar_202502,dbsnp_156,hgmd-public_20204,tsa,e_cited,e_multiple_observations,e_freq,e_topmed,e_hapmap,e_phenotype_or_disease,e_esp,e_gnomad,e_1000g,e_exac,clin_risk_factor,clin_protective,clin_confers_sensitivity,clin_other,clin_drug_response,clin_uncertain_significance,clin_benign,clin_likely_pathogenic,clin_pathogenic,clin_likely_benign,clin_histocompatibility,clin_not_provided,clin_association,ma,maf,mac,aa
str,u32,u32,str,str,str,f64,str,bool,bool,bool,bool,str,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,f32,i32,str
"""1""",10001,10001,"""rs1570391677""","""T""","""A""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10001,10001,"""rs1570391677""","""T""","""C""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10002,10002,"""rs1570391692""","""A""","""C""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10003,10003,"""rs1570391694""","""A""","""C""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null
"""1""",10007,10007,"""rs1639538116""","""T""","""C""",null,"""""",false,false,true,false,"""SNV""",false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,null


In [ ]:
weights = pl.read_database("SELECT id, allele, state, zygosity, weight, rsid, priority, category_id FROM allele_weights", connection=conn)
weights.head(5)

id,allele,state,zygosity,weight,rsid,priority,category_id
i64,str,str,str,f64,str,str,i64
0,"""T""","""alt""","""het""",0.5,"""rs7412""","""1.0""",1
1,"""T""","""alt""","""hom""",1.0,"""rs7412""","""1.0""",1
2,"""C""","""alt""","""het""",-0.5,"""rs429358""","""1.0""",1
3,"""C""","""alt""","""hom""",-1.0,"""rs429358""","""1.0""",1
4,"""G""","""ref""","""hom""",0.97,"""rs5882""","""0.97""",1


In [ ]:
# Get unique rsids from small weights table
rsids = weights["rsid"].unique().to_list()
print(f"Filtering for {len(rsids)} unique rsids")


Filtering for 528 unique rsids


In [ ]:
# Filter the large lazy frame first, then join
joined = df_ensembl.select(
    pl.col("chrom"), 
    pl.col("start"), 
    pl.col("end")
    ).filter(
    pl.col("id").is_in(rsids)
).join(
    weights.lazy(),
    left_on="id",
    right_on="rsid",
    how="inner"
)

In [ ]:
example = joined.head(5).collect(engine="streaming")
example

ColumnNotFoundError: unable to find column "id"; valid columns: ["chrom", "start", "end"]

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
SELECT [col("chrom"), col("start"), col("end")]
  Parquet SCAN [/home/antonkulaga/.cache/just_dna_pipelines/ensembl_variations/splitted_variants/SNV/homo_sapiens-chr1.parquet, ... 24 other sources] [id: 129231707533808]
  PROJECT */40 COLUMNS

In [ ]:
result = joined.collect(engine="streaming")
result

chrom,start,end,id,ref,alt,qual,filter,cosmic_101,clinvar_202502,dbsnp_156,hgmd-public_20204,tsa,e_cited,e_multiple_observations,e_freq,e_topmed,e_hapmap,e_phenotype_or_disease,e_esp,e_gnomad,e_1000g,e_exac,clin_risk_factor,clin_protective,clin_confers_sensitivity,clin_other,clin_drug_response,clin_uncertain_significance,clin_benign,clin_likely_pathogenic,clin_pathogenic,clin_likely_benign,clin_histocompatibility,clin_not_provided,clin_association,ma,maf,mac,aa,id_right,allele,state,zygosity,weight,priority,category_id
str,u32,u32,str,str,str,f64,str,bool,bool,bool,bool,str,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,f32,i32,str,i64,str,str,str,f64,str,i64
"""10""",67891367,67891367,"""rs7896005""","""A""","""G""",null,"""""",false,false,true,false,"""SNV""",true,false,true,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,"""A""",39,"""G""","""alt""","""het""",0.36,"""0.72""",5
"""10""",67891367,67891367,"""rs7896005""","""A""","""T""",null,"""""",false,false,true,false,"""SNV""",true,false,true,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,"""A""",39,"""G""","""alt""","""het""",0.36,"""0.72""",5
"""10""",67891367,67891367,"""rs7896005""","""A""","""G""",null,"""""",false,false,true,false,"""SNV""",true,false,true,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,"""A""",40,"""G""","""alt""","""hom""",0.72,"""0.72""",5
"""10""",67891367,67891367,"""rs7896005""","""A""","""T""",null,"""""",false,false,true,false,"""SNV""",true,false,true,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,"""A""",40,"""G""","""alt""","""hom""",0.72,"""0.72""",5
"""19""",44919689,44919689,"""rs4420638""","""A""","""G""",null,"""""",false,false,true,false,"""SNV""",true,false,true,true,false,true,false,true,true,false,false,false,false,false,false,false,false,false,false,false,false,true,false,"""G""",0.151099,770,"""G""",36,"""G""","""alt""","""het""",-0.375,"""0.75""",1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""12""",57774005,57774005,"""rs10877015""","""A""","""T""",null,"""""",false,false,true,false,"""SNV""",true,false,true,true,false,false,false,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,"""G""",0.355965,1814,"""A""",1009,"""C""","""alt""","""hom""",0.12,"""0.12""",11
"""22""",44203572,44203572,"""rs139170""","""C""","""G""",null,"""""",false,false,true,false,"""SNV""",true,false,true,true,false,false,false,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,"""T""",0.463305,2361,"""T""",990,"""T""","""alt""","""het""",0.065,"""0.13""",0
"""22""",44203572,44203572,"""rs139170""","""C""","""T""",null,"""""",false,false,true,false,"""SNV""",true,false,true,true,false,false,false,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,"""T""",0.463305,2361,"""T""",990,"""T""","""alt""","""het""",0.065,"""0.13""",0


In [ ]:
weights.filter(pl.col("rsid") == "rs7896005")

id,allele,state,zygosity,weight,rsid,priority,category_id
i64,str,str,str,f64,str,str,i64
39,"""G""","""alt""","""het""",0.36,"""rs7896005""","""0.72""",5
40,"""G""","""alt""","""hom""",0.72,"""rs7896005""","""0.72""",5


In [ ]:
result.filter(pl.col("id") == "rs7896005")

chrom,start,end,id,ref,alt,qual,filter,cosmic_101,clinvar_202502,dbsnp_156,hgmd-public_20204,tsa,e_cited,e_multiple_observations,e_freq,e_topmed,e_hapmap,e_phenotype_or_disease,e_esp,e_gnomad,e_1000g,e_exac,clin_risk_factor,clin_protective,clin_confers_sensitivity,clin_other,clin_drug_response,clin_uncertain_significance,clin_benign,clin_likely_pathogenic,clin_pathogenic,clin_likely_benign,clin_histocompatibility,clin_not_provided,clin_association,ma,maf,mac,aa,id_right,allele,state,zygosity,weight,priority,category_id
str,u32,u32,str,str,str,f64,str,bool,bool,bool,bool,str,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,f32,i32,str,i64,str,str,str,f64,str,i64
"""10""",67891367,67891367,"""rs7896005""","""A""","""G""",null,"""""",false,false,true,false,"""SNV""",true,false,true,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,"""A""",39,"""G""","""alt""","""het""",0.36,"""0.72""",5
"""10""",67891367,67891367,"""rs7896005""","""A""","""T""",null,"""""",false,false,true,false,"""SNV""",true,false,true,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,"""A""",39,"""G""","""alt""","""het""",0.36,"""0.72""",5
"""10""",67891367,67891367,"""rs7896005""","""A""","""G""",null,"""""",false,false,true,false,"""SNV""",true,false,true,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,"""A""",40,"""G""","""alt""","""hom""",0.72,"""0.72""",5
"""10""",67891367,67891367,"""rs7896005""","""A""","""T""",null,"""""",false,false,true,false,"""SNV""",true,false,true,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,null,null,null,"""A""",40,"""G""","""alt""","""hom""",0.72,"""0.72""",5


In [ ]:
weights.shape

(1043, 8)

In [ ]:
# Check all tables for columns containing 'rsid' or 'rs' in the name
for table in tables:
    table_name = table[0]
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    rsid_columns = [col[1] for col in columns if 'rs' in col[1].lower() or 'id' in col[1].lower()]
    if rsid_columns:
        print(f"\n{table_name}: {rsid_columns}")

